In [34]:
#%load_ext tensorboard
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorboard.plugins import projector
import json
from collections import defaultdict

In [20]:
data_dir = './data'
train_path = os.path.join(data_dir, 'train.csv')
hero_path = os.path.join(data_dir, 'hero_names.json')

In [66]:
with open('hero_names.json','r') as file:
    hero_names = json.load(file)
pd.DataFrame(hero_names.values())
    
labeled_data = pd.read_csv(train_path)
no_winner = labeled_data['radiant_win'].isna()
labeled_data = labeled_data[~no_winner]


/tmp/ipykernel_2634/1255992683.py:3: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  labeled_data = pd.read_csv(train_path)


In [ ]:
# Convert hero names json to be keyed on id
hero_id_info = {}
for name, hero_dict in hero_names.items():
    this_id = hero_dict['id']
    hero_id_info[this_id] = hero_dict

In [67]:
# Create train test/splits from match data
n_data = len(labeled_data)

random = np.random.RandomState(seed=116)
shuffled_rows = np.arange(n_data)
random.shuffle(shuffled_rows, )

train_frac = 0.8
n_train = int(train_frac*n_data)
n_test = n_data - n_train
train_rows = shuffled_rows[:n_train]
test_rows = shuffled_rows[n_train:]

train_data = labeled_data.iloc[train_rows,:]
test_data = labeled_data.iloc[test_rows,:]


In [68]:
def get_heroes_and_winner(df):
    radiant_cols = [f'r{idx}_hero' for idx in range(1,6)]
    dire_cols = [f'd{idx}_hero' for idx in range(1,6)]


    # make id's start at 0
    radiant_heroes = df[radiant_cols] - 1
    dire_heroes = df[dire_cols] - 1
    winners = df['radiant_win']

    return radiant_heroes, dire_heroes, winners

In [110]:
train_radiant, train_dire, train_y = get_heroes_and_winner(train_data)
test_radiant, test_dire, test_y = get_heroes_and_winner(test_data)

In [111]:
# Calculate historical winrates:
win_counts = defaultdict(lambda: 0)
game_counts = defaultdict(lambda: 0)
for row in range(n_train):
    radiant = train_radiant.iloc[row,:]
    dire = train_dire.iloc[row,:]
    radiant_win = train_y.iloc[row]
    
    for hero in radiant:
        game_counts[hero] += 1
    for hero in dire:
        game_counts[hero] += 1
    
    if radiant_win:
        team = radiant
    else:
        team = dire
        
    for hero in team:
        win_counts[hero] += 1
        
win_rates = {}
for hero in win_counts.keys():
    win_rates[hero] = win_counts[hero]/game_counts[hero]
    

In [112]:
worst_hero = min(win_rates, key=win_rates.get)
best_hero = max(win_rates, key=win_rates.get)
print(f'Best hero is {hero_id_info[best_hero]["localized_name"]} with a winrate of {100*win_rates[best_hero]:.2f}%')
print(f'Worst hero is {hero_id_info[worst_hero]["localized_name"]} with a winrate of {100*win_rates[worst_hero]:.2f}%')

Best hero is Clinkz with a winrate of 59.35%
Worst hero is Night Stalker with a winrate of 39.86%


In [113]:
# Feature of average team winrate

radiant_avg_wr = np.zeros(n_train)
dire_avg_wr = np.zeros(n_train)
test_radiant_wr = np.zeros(n_test)
test_dire_wr = np.zeros(n_test)

for row in range(n_train):
    radiant = train_radiant.iloc[row,:]
    dire = train_dire.iloc[row,:]
    
    radiant_winrates = [win_rates[hero] for hero in radiant]
    dire_winrates = [win_rates[hero] for hero in dire]
    
    radiant_avg = np.mean(radiant_winrates)
    dire_avg = np.mean(dire_winrates)
    
    radiant_avg_wr[row] = radiant_avg
    dire_avg_wr[row] = dire_avg
    
for row in range(n_test):
    radiant = test_radiant.iloc[row,:]
    dire = test_dire.iloc[row,:]
    
    radiant_winrates = [win_rates[hero] for hero in radiant]
    dire_winrates = [win_rates[hero] for hero in dire]
    
    radiant_avg = np.mean(radiant_winrates)
    dire_avg = np.mean(dire_winrates)
    
    test_radiant_wr[row] = radiant_avg
    test_dire_wr[row] = dire_avg
    
    

In [114]:
best_radiant_team = train_radiant.iloc[np.argmax(radiant_avg_wr)]
best_radiant_heros = [hero_id_info[hero]['localized_name'] for hero in best_radiant_team]
worst_radiant_team = train_radiant.iloc[np.argmin(radiant_avg_wr)]
worst_radiant_heros = [hero_id_info[hero]['localized_name'] for hero in worst_radiant_team]
print(f'Best radiant team is {best_radiant_heros} with {np.max(radiant_avg_wr):.2f} wr')
print(f'Did they win? {train_y.iloc[np.argmax(radiant_avg_wr)]}')
print(f'Worst radiant team is {worst_radiant_heros} with {np.min(radiant_avg_wr):.2f} wr')
print(f'Did they win? {train_y.iloc[np.argmin(radiant_avg_wr)]}')

Best radiant team is ['Sniper', 'Windranger', 'Skywrath Mage', 'Clinkz', 'Doom'] with 0.57 wr
Did they win? True
Worst radiant team is ['Phantom Lancer', 'Brewmaster', 'Night Stalker', 'Anti-Mage', 'Bane'] with 0.43 wr
Did they win? False


In [119]:
class EmbeddingModel(tf.keras.Model):
    def __init__(self, pool_size=123, embedding_size=32, team_size=5,
                 n_hidden_predictor=128, dropout_rate=0.1):
        super(EmbeddingModel, self).__init__()

        self.embedding = tf.keras.layers.Embedding(pool_size, embedding_size,
                                                   input_length=team_size)

        self.predictor = tf.keras.Sequential(
            [
             tf.keras.layers.InputLayer(input_shape=(embedding_size*2 + 2,)),
             tf.keras.layers.Dropout(dropout_rate),
             tf.keras.layers.Dense(units=n_hidden_predictor, activation=tf.nn.tanh),
             tf.keras.layers.Dense(units=n_hidden_predictor, activation=tf.nn.tanh),
             tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
            ]
            )
        return


    def call(self, inputs):
        radiant, dire, radiant_wr, dire_wr = inputs

        radiant_embedding = self.embedding(radiant)
        dire_embedding = self.embedding(dire)

        radiant_embedding_sum = tf.reduce_sum(radiant_embedding, axis=1)
        dire_embedding_sum = tf.reduce_sum(dire_embedding, axis=1)

        pred_inputs = tf.concat((radiant_embedding_sum, dire_embedding_sum, radiant_wr, dire_wr), axis=-1)
        prediction = self.predictor(pred_inputs)

        return prediction


In [140]:
model = EmbeddingModel(pool_size=112, embedding_size=32, dropout_rate=0.1)
loss = tf.keras.losses.BinaryCrossentropy()

log_dir='./logs/embedding/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir)]

2022-04-22 23:37:02.137613: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-22 23:37:02.137643: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-22 23:37:02.137690: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [141]:
optimizer =  tf.keras.optimizers.Adam(1e-4)
model.compile(optimizer=optimizer, loss=loss,metrics=[tf.metrics.BinaryAccuracy()])

In [147]:
model.fit(x=[tf.cast(train_radiant,dtype=tf.int32), tf.cast(train_dire, dtype=tf.int32),
             tf.cast(radiant_avg_wr, dtype=tf.float32), tf.cast(dire_avg_wr, dtype=tf.float32)],
          y=tf.cast(train_y.astype(int), dtype=tf.float32),
          callbacks=callbacks, shuffle=True,
          batch_size=8196, epochs=20000,
          validation_data=
          ([tf.cast(test_radiant,dtype=tf.int32), tf.cast(test_dire, dtype=tf.int32),
             tf.cast(test_radiant_wr, dtype=tf.float32), tf.cast(test_dire_wr, dtype=tf.float32)],
            tf.cast(test_y.astype(int), dtype=tf.float32)))

Epoch 1/20000
 6/40 [===>..........................] - ETA: 1s - loss: 0.6003 - binary_accuracy: 0.6729

2022-04-23 02:56:23.090809: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-23 02:56:23.090878: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-23 02:56:23.141038: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-04-23 02:56:23.142951: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-04-23 02:56:23.150509: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/embedding/train/plugins/profile/2022_04_23_02_56_23
2022-04-23 02:56:23.153519: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./logs/embedding/train/plugins/profile/2022_04_23_02_56_23/MrChipsNVME.trace.json.gz
2022-04-23 02:56:23.159995: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./logs/embedding/train/plugins/profile/2022_04_23_02_56

40/40 [==============================] - 1s 24ms/step - loss: 0.6006 - binary_accuracy: 0.6732 - val_loss: 0.7085 - val_binary_accuracy: 0.5771
Epoch 2/20000
40/40 [==============================] - 1s 13ms/step - loss: 0.5999 - binary_accuracy: 0.6750 - val_loss: 0.7084 - val_binary_accuracy: 0.5768
Epoch 3/20000
40/40 [==============================] - 1s 15ms/step - loss: 0.6006 - binary_accuracy: 0.6728 - val_loss: 0.7086 - val_binary_accuracy: 0.5766
Epoch 4/20000
40/40 [==============================] - 1s 18ms/step - loss: 0.6013 - binary_accuracy: 0.6724 - val_loss: 0.7084 - val_binary_accuracy: 0.5766
Epoch 5/20000
40/40 [==============================] - 1s 16ms/step - loss: 0.6002 - binary_accuracy: 0.6736 - val_loss: 0.7088 - val_binary_accuracy: 0.5777
Epoch 6/20000
40/40 [==============================] - 1s 17ms/step - loss: 0.6005 - binary_accuracy: 0.6737 - val_loss: 0.7087 - val_binary_accuracy: 0.5767
Epoch 7/20000
32/40 [=======================>......] - ETA: 0s - l

KeyboardInterrupt: 

In [143]:
# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for id in range(1,113):
    if id in hero_id_info.keys():
        hero_name = hero_id_info[id]['localized_name']
        f.write(f'{hero_name}\n')
    else:
        f.write('Unknown\n')


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model.embedding.get_weights()[0])
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

'./logs/embedding/embedding.ckpt-1'

In [102]:
id_he

{'npc_dota_hero_antimage': {'id': 1,
  'name': 'npc_dota_hero_antimage',
  'localized_name': 'Anti-Mage',
  'primary_attr': 'agi',
  'attack_type': 'Melee',
  'roles': ['Carry', 'Escape', 'Nuker'],
  'img': '/apps/dota2/images/heroes/antimage_full.png?',
  'icon': '/apps/dota2/images/heroes/antimage_icon.png',
  'base_health': 200,
  'base_health_regen': 0.25,
  'base_mana': 75,
  'base_mana_regen': 0,
  'base_armor': -1,
  'base_mr': 25,
  'base_attack_min': 29,
  'base_attack_max': 33,
  'base_str': 23,
  'base_agi': 24,
  'base_int': 12,
  'str_gain': 1.3,
  'agi_gain': 3.2,
  'int_gain': 1.8,
  'attack_range': 150,
  'projectile_speed': 0,
  'attack_rate': 1.4,
  'move_speed': 310,
  'turn_rate': 0.5,
  'cm_enabled': True,
  'legs': 2},
 'npc_dota_hero_axe': {'id': 2,
  'name': 'npc_dota_hero_axe',
  'localized_name': 'Axe',
  'primary_attr': 'str',
  'attack_type': 'Melee',
  'roles': ['Initiator', 'Durable', 'Disabler', 'Jungler'],
  'img': '/apps/dota2/images/heroes/axe_full.png

In [65]:
n_train

327835

,id,name,localized_name,primary_attr,attack_type,roles,img,icon,base_health,base_health_regen,...,str_gain,agi_gain,int_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,cm_enabled,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",/apps/dota2/images/heroes/antimage_full.png?,/apps/dota2/images/heroes/antimage_icon.png,200,0.25,...,1.3,3.2,1.8,150,0,1.4,310,0.5,True,2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler]",/apps/dota2/images/heroes/axe_full.png?,/apps/dota2/images/heroes/axe_icon.png,200,2.75,...,3.4,2.2,1.6,150,900,1.7,295,0.6,True,2
2,3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",/apps/dota2/images/heroes/bane_full.png?,/apps/dota2/images/heroes/bane_icon.png,200,NaN,...,2.6,2.6,2.6,400,900,1.7,305,0.6,True,4
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Jungler, Nuker, Initiator]",/apps/dota2/images/heroes/bloodseeker_full.png?,/apps/dota2/images/heroes/bloodseeker_icon.png,200,NaN,...,2.7,3.5,1.7,150,900,1.7,295,0.5,True,2
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker, Jungler]",/apps/dota2/images/heroes/crystal_maiden_full....,/apps/dota2/images/heroes/crystal_maiden_icon.png,200,NaN,...,2.2,1.6,3.3,600,900,1.7,275,0.5,True,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,114,npc_dota_hero_monkey_king,Monkey King,agi,Melee,"[Carry, Escape, Disabler, Initiator]",/apps/dota2/images/heroes/monkey_king_full.png?,/apps/dota2/images/heroes/monkey_king_icon.png,200,1.50,...,2.8,3.7,1.8,300,900,1.7,305,0.6,True,2
113,119,npc_dota_hero_dark_willow,Dark Willow,int,Ranged,"[Support, Nuker, Disabler, Escape]",/apps/dota2/images/heroes/dark_willow_full.png?,/apps/dota2/images/heroes/dark_willow_icon.png,200,NaN,...,2.0,1.6,3.5,475,1200,1.3,290,0.7,True,2
114,120,npc_dota_hero_pangolier,Pangolier,agi,Melee,"[Carry, Nuker, Disabler, Durable, Escape, Init...",/apps/dota2/images/heroes/pangolier_full.png?,/apps/dota2/images/heroes/pangolier_icon.png,200,0.25,...,2.5,3.2,1.9,150,900,1.7,305,1.0,True,2
115,121,npc_dota_hero_grimstroke,Grimstroke,int,Ranged,"[Support, Nuker, Disabler, Escape]",/apps/dota2/images/heroes/grimstroke_full.png?,/apps/dota2/images/heroes/grimstroke_icon.png,200,NaN,...,2.4,1.9,3.8,550,900,1.7,290,0.6,True,2
